In [2]:
%maven ai.djl:api:0.28.0


In [3]:
%maven ai.djl:basicdataset:0.28.0
%maven ai.djl:model-zoo:0.28.0
%maven ai.djl.mxnet:mxnet-engine:0.28.0
%maven org.slf4j:slf4j-simple:1.7.36

In [4]:
import java.nio.file.*;

import ai.djl.*;
import ai.djl.basicdataset.cv.classification.Mnist;
import ai.djl.ndarray.types.*;
import ai.djl.training.*;
import ai.djl.training.dataset.*;
import ai.djl.training.initializer.*;
import ai.djl.training.loss.*;
import ai.djl.training.listener.*;
import ai.djl.training.evaluator.*;
import ai.djl.training.optimizer.*;
import ai.djl.training.util.*;
import ai.djl.basicmodelzoo.cv.classification.*;
import ai.djl.basicmodelzoo.basic.*;

In [5]:
import ai.djl.*;
import ai.djl.nn.*;
import ai.djl.nn.core.*;
import ai.djl.training.*;

In [17]:
%maven ai.djl:api:0.28.0
%maven ai.djl:model-zoo:0.28.0
%maven ai.djl.mxnet:mxnet-engine:0.28.0
%maven ai.djl.mxnet:mxnet-model-zoo:0.28.0
%maven org.slf4j:slf4j-simple:1.7.36

In [18]:
import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import java.util.stream.*;
import ai.djl.*;
import ai.djl.basicmodelzoo.basic.*;
import ai.djl.ndarray.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.util.NDImageUtils;
import ai.djl.translate.*;

In [6]:
Application application = Application.CV.IMAGE_CLASSIFICATION;


In [7]:
long inputSize = 28*28;
long outputSize = 10;

In [8]:
SequentialBlock block = new SequentialBlock();

In [9]:
block.add(Blocks.batchFlattenBlock(inputSize));
block.add(Linear.builder().setUnits(128).build());
block.add(Activation::relu);
block.add(Linear.builder().setUnits(64).build());
block.add(Activation::relu);
block.add(Linear.builder().setUnits(outputSize).build());

block

SequentialBlock {
	batchFlatten
	Linear
	LambdaBlock
	Linear
	LambdaBlock
	Linear
}

In [10]:
int batchSize = 32;
Mnist mnist = Mnist.builder().setSampling(batchSize, true).build();
mnist.prepare(new ProgressBar());

In [11]:
Model model = Model.newInstance("mlp");
model.setBlock(new Mlp(28 * 28, 10, new int[] {128, 64}));

In [12]:
DefaultTrainingConfig config = new DefaultTrainingConfig(Loss.softmaxCrossEntropyLoss())
    //softmaxCrossEntropyLoss is a standard loss for classification problems
    .addEvaluator(new Accuracy()) // Use accuracy so we humans can understand how accurate the model is
    .addTrainingListeners(TrainingListener.Defaults.logging());

// Now that we have our training configuration, we should create a new trainer for our model
Trainer trainer = model.newTrainer(config);

[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Training on: cpu().
[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Load MXNet Engine Version 1.9.0 in 0.526 ms.


In [13]:
trainer.initialize(new Shape(1, 28 * 28));


In [14]:
// Deep learning is typically trained in epochs where each epoch trains the model on each item in the dataset once.
int epoch = 2;

EasyTrain.fit(trainer, epoch, mnist, null);

Training:    100% |████████████████████████████████████████| 


[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Epoch 1 finished.


Training:    100% |████████████████████████████████████████| 


[IJava-executor-0] INFO ai.djl.training.listener.LoggingTrainingListener - Epoch 2 finished.


In [16]:
Path modelDir = Paths.get("/Users/prakashr/Documents/codebase/java-jupyter-examples");
Files.createDirectories(modelDir);

model.setProperty("Epoch", String.valueOf(epoch));

model.save(modelDir, "mlp");

model

Model (
	Name: mlp
	Model location: /Users/prakashr/Documents/codebase/java-jupyter-examples
	Data Type: float32
	Epoch: 2
)

In [19]:
var img = ImageFactory.getInstance().fromUrl("https://resources.djl.ai/images/0.png");
img.getWrappedImage();

In [20]:
Path modelDir = Paths.get("/Users/prakashr/Documents/codebase/java-jupyter-examples");
Model model = Model.newInstance("mlp");
model.setBlock(new Mlp(28 * 28, 10, new int[] {128, 64}));
model.load(modelDir);

In [22]:
Translator<Image, Classifications> translator = new Translator<Image, Classifications>() {
    @Override
    public NDList processInput(TranslatorContext ctx, Image input) {
        // Convert Image to NDArray
        NDArray array = input.toNDArray(ctx.getNDManager(), Image.Flag.GRAYSCALE);
        return new NDList(NDImageUtils.toTensor(array));
    }
    
    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        // Create a Classifications with the output probabilities
        NDArray probabilities = list.singletonOrThrow().softmax(0);
        List<String> classNames = IntStream.range(0, 10)
                                           .mapToObj(String::valueOf)
                                           .collect(Collectors.toList());
        return new Classifications(classNames, probabilities);
    }
    
    @Override
    public Batchifier getBatchifier() {
        return Batchifier.STACK;
    }
};

In [23]:
var predictor = model.newPredictor(translator);


In [24]:
var classifications = predictor.predict(img);

classifications

[
	{"class": "0", "probability": 0.99949}
	{"class": "2", "probability": 0.00048}
	{"class": "6", "probability": 0.00001}
	{"class": "5", "probability": 8.2e-07}
	{"class": "1", "probability": 7.1e-07}
]